In [1]:
import sys
from  SPARQLWrapper import SPARQLWrapper

In [2]:
# SPARQLエンドポイントのURLを指定してインスタンスを作成する。
sparql = SPARQLWrapper('http://ja.dbpedia.org/sparql')

In [12]:
sparql.setQuery(r'''
    SELECT * WHERE {
    ?s rdf:type dbpedia-owl:Museum ;
      prop-ja:所在地 ?address .
    FILTER REGEX(?address, "^\\p{Han}{2,3}[都道府県]")
    } ORDER BY ?s
''') 

In [13]:
# 取得するフォーマットとしてjsonを指定
sparql.setReturnFormat('json')

In [14]:
# queryを実行し、convert()でレスポンスをパースしてdictを得る。
response = sparql.query().convert()

In [25]:
len(response['results']['bindings'])

1577

In [39]:
sparql.setQuery(r'''
    SELECT * WHERE{
        ?s rdf:type dbpedia-owl:Museum ;
        prop-ja:所在地 ?address .
        OPTIONAL { ?s rdfs:label ?label . }
        OPTIONAL {
        ?s prop-ja:経度度 ?lon_degree ;
            prop-ja:経度分 ?lon_minute ;
            prop-ja:経度秒 ?lon_second ;
            prop-ja:緯度度 ?lat_degree ;
            prop-ja:経度分 ?lat_minute ;
            prop-ja:経度秒 ?lat_second .
        }
        FILTER REGEX(?address, "^\\p{Han}{2,3}[都道府県]")
    } ORDER BY ?s
    ''')

In [40]:
sparql.setReturnFormat('json')
response = sparql.query().convert()

In [48]:
# resultを加工して辞書作成
# 反復処理できる用にyieldで本当はやる。
a = {name:binding['value'] for name, binding in response['results']['bindings'][0].items()}

In [86]:
a['address']

'兵庫県神戸市灘区岩屋中町4丁目2番7号'

In [87]:
if 'lon_degree' in a:
    print('hello')

hello


In [88]:
a

{'address': '兵庫県神戸市灘区岩屋中町4丁目2番7号',
 'label': 'BBプラザ美術館',
 'lat_degree': '34',
 'lat_minute': '13',
 'lat_second': '4',
 'lon_degree': '135',
 'lon_minute': '13',
 'lon_second': '4',
 's': 'http://ja.dbpedia.org/resource/BBプラザ美術館'}

In [51]:
# yahooAPIでジオコーディングする。
YAHOO_GEOCODER_API_URL = 'http://geo.search.olp.yahooapis.jp/OpenLocalPlatform/V1/geoCoder'

In [65]:
import pandas as pd
from urllib.request import urlopen
from urllib.parse import urlencode

In [62]:
yahoo_api = pd.read_csv('yahoo_api.csv', header=None, names=['name','value'])
token = yahoo_api.loc[0, 'value']

In [68]:
url = YAHOO_GEOCODER_API_URL + '?' + urlencode({
    'appid':token,
    'output':'json',
    'query':a['address']
})

In [69]:
response_text = urlopen(url).read()

In [78]:
import json
coordinates = json.loads(response_text.decode('utf-8'))

In [81]:
coordinates['Feature'][0]['Geometry']['Coordinates'].split(',')

['135.21779210', '34.70326468']

In [82]:
coordinates

{'Feature': [{'Category': [],
   'Description': '',
   'Geometry': {'BoundingBox': '135.21712300,34.70254200 135.21992100,34.70438400',
    'Coordinates': '135.21779210,34.70326468',
    'Type': 'point'},
   'Gid': '',
   'Id': '28102.9.4.2.7',
   'Name': '兵庫県神戸市灘区岩屋中町4丁目2-7',
   'Property': {'Address': '兵庫県神戸市灘区岩屋中町4丁目2-7',
    'AddressMatchingLevel': '6',
    'AddressType': '地番・戸番',
    'CassetteId': 'b22fee69b0dcaf2c2fe2d6a27906dafc',
    'Country': {'Code': 'JP', 'Name': '日本'},
    'GovernmentCode': '28102',
    'Uid': 'efb166aa75eae60aa5492dba1d5737ebe2460ae6',
    'Yomi': 'ヒョウゴケンコウベシナダクイワヤナカマチ4チョウメ'},
   'Style': []}],
 'ResultInfo': {'Copyright': '',
  'Count': 1,
  'Description': '',
  'Latency': 0.083,
  'Start': 1,
  'Status': 200,
  'Total': 1}}